In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
unit_all = pd.read_csv("/content/drive/Shareddrives/Multicampus/Preprocessing/unit_all.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,6,7,11,12) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
unit_all = unit_all[['SUBJECT_ID','ITEMID','CHARTTIME','VALUE','VALUENUM','VALUEUOM','LABEL']]

# 원하는 검사 추출

In [ ]:
itemid_list = [51279, 50820, 50971,50868,50882,50902,51301,50983,50912,50970,
               51006,51446,51200,51222,51254,51256,51265,51275,
               51221,50931,50960,50893]

In [ ]:
unit_all = unit_all[unit_all['ITEMID'].isin(itemid_list)]
unit_all.head()

SUBJECT_ID  ITEMID            CHARTTIME  ... VALUENUM  VALUEUOM      LABEL
937103          22   50868  2196-04-09 10:14:00  ...     13.0     mEq/L  Anion Gap
937104          22   50868  2196-04-10 08:01:00  ...     13.0     mEq/L  Anion Gap
937105          24   50868  2139-06-06 17:54:00  ...     14.0     mEq/L  Anion Gap
937106          24   50868  2139-06-07 03:00:00  ...     14.0     mEq/L  Anion Gap
937107          24   50868  2139-06-08 06:05:00  ...     15.0     mEq/L  Anion Gap

[5 rows x 7 columns]

In [ ]:
len(unit_all['SUBJECT_ID'].unique())  #37955명의 환자

37955

# 특정 시간 내에 일어난 검사만 추출

## admission time 포함된 데이터 생성

In [ ]:
adm=pd.read_csv('/content/drive/Shareddrives/Multicampus/mimic-iii-clinical-database-1.4/ADMISSIONS.csv.gz',
                parse_dates=['ADMITTIME'])

In [ ]:
adm.head()

ROW_ID  SUBJECT_ID  ...  HOSPITAL_EXPIRE_FLAG HAS_CHARTEVENTS_DATA
0      21          22  ...                     0                    1
1      22          23  ...                     0                    1
2      23          23  ...                     0                    1
3      24          24  ...                     0                    1
4      25          25  ...                     0                    1

[5 rows x 19 columns]

In [ ]:
dummy=pd.read_csv('/content/drive/Shareddrives/Multicampus/Preprocessing/0414_prepro_dummy.csv')

In [ ]:
dummy

SUBJECT_ID  AGE  staytime  ... ETH_WHITE  GEN_F  GEN_M
0              22   64         2  ...         1      1      0
1              24   39         3  ...         1      0      1
2              25   58         4  ...         1      0      1
3              26   72         7  ...         0      0      1
4              28   74         6  ...         1      0      1
...           ...  ...       ...  ...       ...    ...    ...
30919       98794   78         5  ...         1      0      1
30920       98797   88         1  ...         1      0      1
30921       98800   19         3  ...         1      1      0
30922       98802   83         1  ...         1      1      0
30923       98805   42         9  ...         1      0      1

[30924 rows x 67 columns]

In [ ]:
unit_all_time = pd.merge(unit_all, dummy['SUBJECT_ID'], how='left', on='SUBJECT_ID')

In [ ]:
len(unit_all_time['SUBJECT_ID'].unique())   #37955

37955

In [ ]:
unit_all_time.head()

SUBJECT_ID  ITEMID            CHARTTIME VALUE  VALUENUM VALUEUOM      LABEL
0          22   50868  2196-04-09 10:14:00    13      13.0    mEq/L  Anion Gap
1          22   50868  2196-04-10 08:01:00    13      13.0    mEq/L  Anion Gap
2          24   50868  2139-06-06 17:54:00    14      14.0    mEq/L  Anion Gap
3          24   50868  2139-06-07 03:00:00    14      14.0    mEq/L  Anion Gap
4          24   50868  2139-06-08 06:05:00    15      15.0    mEq/L  Anion Gap

In [ ]:
unit_all_time=unit_all_time.merge(adm[['SUBJECT_ID','ADMITTIME']], on='SUBJECT_ID', how='left')
unit_all_time.head()

SUBJECT_ID  ITEMID  ...      LABEL           ADMITTIME
0          22   50868  ...  Anion Gap 2196-04-09 12:26:00
1          22   50868  ...  Anion Gap 2196-04-09 12:26:00
2          24   50868  ...  Anion Gap 2139-06-06 16:14:00
3          24   50868  ...  Anion Gap 2139-06-06 16:14:00
4          24   50868  ...  Anion Gap 2139-06-06 16:14:00

[5 rows x 8 columns]

## chart time - admit time 날짜, 시간 계산

In [ ]:
from datetime import datetime, timedelta

In [ ]:
# 24hrs = ADMITTIME + 24시간 
unit_all_time['24hrs']=unit_all_time.ADMITTIME.apply(lambda x:x + timedelta(days=0, hours=24, minutes=0))

In [ ]:
unit_all_time.head()

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
0          22   50868  ... 2196-04-09 12:26:00 2196-04-10 12:26:00
1          22   50868  ... 2196-04-09 12:26:00 2196-04-10 12:26:00
2          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
3          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
4          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00

[5 rows x 9 columns]

In [ ]:
unit_all_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6340692 entries, 0 to 6340691
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   ITEMID      int64         
 2   CHARTTIME   object        
 3   VALUE       object        
 4   VALUENUM    float64       
 5   VALUEUOM    object        
 6   LABEL       object        
 7   ADMITTIME   datetime64[ns]
 8   24hrs       datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(2), object(4)
memory usage: 483.8+ MB


## 입원 후 24시간 내의 검사결과 데이터 추출

In [ ]:
unit_all_time['CHARTTIME'] = pd.to_datetime(unit_all_time['CHARTTIME'])

In [ ]:
unit_all_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6340692 entries, 0 to 6340691
Data columns (total 9 columns):
 #   Column      Dtype         
---  ------      -----         
 0   SUBJECT_ID  int64         
 1   ITEMID      int64         
 2   CHARTTIME   datetime64[ns]
 3   VALUE       object        
 4   VALUENUM    float64       
 5   VALUEUOM    object        
 6   LABEL       object        
 7   ADMITTIME   datetime64[ns]
 8   24hrs       datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2), object(3)
memory usage: 483.8+ MB


In [ ]:
# CHARTTIME이 ADMITTIME 보다 크고 ADMITTIME + 24시간 보다 작은 행만 추출
df_24hrs = unit_all_time[(unit_all_time['CHARTTIME'] < unit_all_time['24hrs']) & (unit_all_time['CHARTTIME'] >= unit_all_time['ADMITTIME'])]
df_24hrs.head()

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
1          22   50868  ... 2196-04-09 12:26:00 2196-04-10 12:26:00
2          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
3          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
7          25   50868  ... 2160-11-02 02:06:00 2160-11-03 02:06:00
8          25   50868  ... 2160-11-02 02:06:00 2160-11-03 02:06:00

[5 rows x 9 columns]

In [ ]:
len(df_24hrs['SUBJECT_ID'].unique())  #37955명 => 36439명

36439

In [ ]:
df_24hrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043205 entries, 1 to 6340688
Data columns (total 9 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1043205 non-null  int64         
 1   ITEMID      1043205 non-null  int64         
 2   CHARTTIME   1043205 non-null  datetime64[ns]
 3   VALUE       1043200 non-null  object        
 4   VALUENUM    1042798 non-null  float64       
 5   VALUEUOM    1043205 non-null  object        
 6   LABEL       1043205 non-null  object        
 7   ADMITTIME   1043205 non-null  datetime64[ns]
 8   24hrs       1043205 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2), object(3)
memory usage: 79.6+ MB


### object형으로 작성된 데이터 -> num 데이터로 변경

In [ ]:
df_24hrs_object = df_24hrs[(df_24hrs['VALUE'].isna()==False) & (df_24hrs['VALUENUM'].isna()==True)]

In [ ]:
df_24hrs_object

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
358640          430   50882  ... 2155-09-16 00:14:00 2155-09-17 00:14:00
366128         1496   50882  ... 2134-01-02 15:10:00 2134-01-03 15:10:00
368117         2112   50882  ... 2164-12-17 00:56:00 2164-12-18 00:56:00
383458         4447   50882  ... 2168-03-24 18:56:00 2168-03-25 18:56:00
393675         5833   50882  ... 2179-07-26 02:28:00 2179-07-27 02:28:00
...             ...     ...  ...                 ...                 ...
6306787       77416   50820  ... 2103-01-08 07:15:00 2103-01-09 07:15:00
6312752       91399   50820  ... 2117-05-20 19:24:00 2117-05-21 19:24:00
6320425       86233   50820  ... 2177-11-26 08:00:00 2177-11-27 08:00:00
6331072       98615   50820  ... 2196-10-30 06:47:00 2196-10-31 06:47:00
6337793       92314   50820  ... 2155-03-31 15:28:00 2155-04-01 15:28:00

[402 rows x 9 columns]

In [ ]:
df_24hrs_object['VALUE'].unique()

array(['LESS THAN 5', 'QNS TO PERFORM TEST', 'QNS TO PERFORM',
       'GREATER THAN 50', 'ERROR', 'QNS', '<5',
       'VERIFIED BY REPLICATE ANALYSIS',
       'COMPUTER NETWORK FAILURE. TEST NOT RESULTED.', 'QNS TO RUN',
       'QNS TO RUN, NOTIFIED CHRISTINE', 'LESS THAN 5.0',
       'GREATER THAN 140', 'LESS THAN 60', '>140', '<0.1',
       'LESS THAN 0.1', 'GREATER THAN 1300', 'LESS THAN 0.3', '<0.3',
       'LESS THAN 1.0', 'GREATER THAN 10', 'GREATER THAN 10.0',
       'HEMOLYSIS FALSELY ELEVATES K', '>10', '>10.0', 'BELOW 80',
       'GREATER THAN 180', 'UNABLE TO MEASURE ACCURATELY',
       'UNABLE TO PERFORM', 'ERROR DISREGARD PREVIOUS RESULT OF 30.4',
       'UNABLE TO REPORT', 'DISREGARD RESULT',
       'DISREGARD RESULT, PREVIOUSLY REPORTED AS 9.5',
       'DISREGARD RESULT, PREVIOUSLY REPORTED AS 10.2',
       'ERROR DISREGARD PREVIOUS RESULT OF 10.7',
       'UNABLE TO REPORT DUE TO LIPEMIA',
       'UNABLE TO REPORT DUE TO PLTS CLUMPS',
       'UNABLE TO REPORT PLATELET C

In [ ]:
def get_num(x):
    return float("".join(ele for ele in x if ele.isdigit() or ele == "."))

In [ ]:
range(len(df_24hrs_object))

range(0, 402)

In [ ]:
for i in range(len(df_24hrs_object)):
    try:
        df_24hrs_object.iloc[i,4]=get_num(df_24hrs_object.iloc[i,3])  #value열에서 get_num을 한 결과를 valuenum열로 넣어줌
    except:
        pass

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
# 처리결과 확인1
df_24hrs_object.head()

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
358640         430   50882  ... 2155-09-16 00:14:00 2155-09-17 00:14:00
366128        1496   50882  ... 2134-01-02 15:10:00 2134-01-03 15:10:00
368117        2112   50882  ... 2164-12-17 00:56:00 2164-12-18 00:56:00
383458        4447   50882  ... 2168-03-24 18:56:00 2168-03-25 18:56:00
393675        5833   50882  ... 2179-07-26 02:28:00 2179-07-27 02:28:00

[5 rows x 9 columns]

In [ ]:
# 처리결과 확인2
df_24hrs_object.tail()

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
6306787       77416   50820  ... 2103-01-08 07:15:00 2103-01-09 07:15:00
6312752       91399   50820  ... 2117-05-20 19:24:00 2117-05-21 19:24:00
6320425       86233   50820  ... 2177-11-26 08:00:00 2177-11-27 08:00:00
6331072       98615   50820  ... 2196-10-30 06:47:00 2196-10-31 06:47:00
6337793       92314   50820  ... 2155-03-31 15:28:00 2155-04-01 15:28:00

[5 rows x 9 columns]

In [ ]:
df_24hrs_object.info()  #nan 127개 => error로 인해 수치로 바꿀수 없는 object

<class 'pandas.core.frame.DataFrame'>
Int64Index: 402 entries, 358640 to 6337793
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   SUBJECT_ID  402 non-null    int64         
 1   ITEMID      402 non-null    int64         
 2   CHARTTIME   402 non-null    datetime64[ns]
 3   VALUE       402 non-null    object        
 4   VALUENUM    226 non-null    float64       
 5   VALUEUOM    402 non-null    object        
 6   LABEL       402 non-null    object        
 7   ADMITTIME   402 non-null    datetime64[ns]
 8   24hrs       402 non-null    datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2), object(3)
memory usage: 31.4+ KB


#### num으로 바꾼 데이터 원래 데이터 프레임에 넣어주기

In [ ]:
valuenum = pd.merge(df_24hrs[['SUBJECT_ID','ITEMID','CHARTTIME','VALUENUM']], 
                    df_24hrs_object[['SUBJECT_ID','ITEMID','CHARTTIME','VALUENUM']],
                    on=['SUBJECT_ID','ITEMID','CHARTTIME'],
                    how='left')

In [ ]:
valuenum.head()

SUBJECT_ID  ITEMID           CHARTTIME  VALUENUM_x  VALUENUM_y
0          22   50868 2196-04-10 08:01:00        13.0         NaN
1          24   50868 2139-06-06 17:54:00        14.0         NaN
2          24   50868 2139-06-07 03:00:00        14.0         NaN
3          25   50868 2160-11-02 03:59:00        19.0         NaN
4          25   50868 2160-11-02 06:56:00        12.0         NaN

In [ ]:
valuenum.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043205 entries, 0 to 1043204
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1043205 non-null  int64         
 1   ITEMID      1043205 non-null  int64         
 2   CHARTTIME   1043205 non-null  datetime64[ns]
 3   VALUENUM_x  1042798 non-null  float64       
 4   VALUENUM_y  226 non-null      float64       
dtypes: datetime64[ns](1), float64(2), int64(2)
memory usage: 47.8 MB


In [ ]:
df_24hrs['RESULT'] = np.where(valuenum['VALUENUM_x'].isna(), valuenum['VALUENUM_y'], valuenum['VALUENUM_x'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_24hrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043205 entries, 1 to 6340688
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1043205 non-null  int64         
 1   ITEMID      1043205 non-null  int64         
 2   CHARTTIME   1043205 non-null  datetime64[ns]
 3   VALUE       1043200 non-null  object        
 4   VALUENUM    1042798 non-null  float64       
 5   VALUEUOM    1043205 non-null  object        
 6   LABEL       1043205 non-null  object        
 7   ADMITTIME   1043205 non-null  datetime64[ns]
 8   24hrs       1043205 non-null  datetime64[ns]
 9   RESULT      1043024 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(2), object(3)
memory usage: 87.5+ MB


### 아예 결측값인 경우(value와 valuenum 모두 NaN값)

In [ ]:
all_nan_idx = df_24hrs[(df_24hrs['VALUE'].isna()==True) & (df_24hrs['VALUENUM'].isna()==True)].index

In [ ]:
all_nan_idx

Int64Index([55108, 145008, 305586, 3872503, 5176175], dtype='int64')

In [ ]:
df_24hrs = df_24hrs.drop(all_nan_idx)

In [ ]:
# 확인
df_24hrs[(df_24hrs['VALUE'].isna()==True) & (df_24hrs['VALUENUM'].isna()==True)]
# 없음 -> 잘 제거됨

Empty DataFrame
Columns: [SUBJECT_ID, ITEMID, CHARTTIME, VALUE, VALUENUM, VALUEUOM, LABEL, ADMITTIME, 24hrs, RESULT]
Index: []

### 최종적으로 정리한 24시간 데이터 저장

In [ ]:
df_24hrs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043200 entries, 1 to 6340688
Data columns (total 10 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1043200 non-null  int64         
 1   ITEMID      1043200 non-null  int64         
 2   CHARTTIME   1043200 non-null  datetime64[ns]
 3   VALUE       1043200 non-null  object        
 4   VALUENUM    1042798 non-null  float64       
 5   VALUEUOM    1043200 non-null  object        
 6   LABEL       1043200 non-null  object        
 7   ADMITTIME   1043200 non-null  datetime64[ns]
 8   24hrs       1043200 non-null  datetime64[ns]
 9   RESULT      1043024 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(2), object(3)
memory usage: 87.5+ MB


In [ ]:
df_24hrs_final = df_24hrs[['SUBJECT_ID','ITEMID','LABEL','RESULT','VALUEUOM','CHARTTIME','ADMITTIME','24hrs']]

In [ ]:
df_24hrs_final.head()

SUBJECT_ID  ITEMID  ...           ADMITTIME               24hrs
1          22   50868  ... 2196-04-09 12:26:00 2196-04-10 12:26:00
2          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
3          24   50868  ... 2139-06-06 16:14:00 2139-06-07 16:14:00
7          25   50868  ... 2160-11-02 02:06:00 2160-11-03 02:06:00
8          25   50868  ... 2160-11-02 02:06:00 2160-11-03 02:06:00

[5 rows x 8 columns]

In [ ]:
df_24hrs_final.info()  #127개 결측값

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043200 entries, 1 to 6340688
Data columns (total 8 columns):
 #   Column      Non-Null Count    Dtype         
---  ------      --------------    -----         
 0   SUBJECT_ID  1043200 non-null  int64         
 1   ITEMID      1043200 non-null  int64         
 2   LABEL       1043200 non-null  object        
 3   RESULT      1043024 non-null  float64       
 4   VALUEUOM    1043200 non-null  object        
 5   CHARTTIME   1043200 non-null  datetime64[ns]
 6   ADMITTIME   1043200 non-null  datetime64[ns]
 7   24hrs       1043200 non-null  datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(2), object(2)
memory usage: 71.6+ MB


In [ ]:
df_24hrs_final.to_csv('/content/drive/Shareddrives/Multicampus/Preprocessing/unit_num_24hrs.csv', index=False)

## 입원 후 12시간 내의 검사결과 데이터 추출

In [ ]:
df_24hrs_final['hours'] = (df_24hrs_final.CHARTTIME - df_24hrs_final.ADMITTIME).apply(lambda x: x.seconds/3600).round(2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
df_24hrs_final.head()

SUBJECT_ID  ITEMID      LABEL  ...           ADMITTIME               24hrs  hours
1          22   50868  Anion Gap  ... 2196-04-09 12:26:00 2196-04-10 12:26:00  19.58
2          24   50868  Anion Gap  ... 2139-06-06 16:14:00 2139-06-07 16:14:00   1.67
3          24   50868  Anion Gap  ... 2139-06-06 16:14:00 2139-06-07 16:14:00  10.77
7          25   50868  Anion Gap  ... 2160-11-02 02:06:00 2160-11-03 02:06:00   1.88
8          25   50868  Anion Gap  ... 2160-11-02 02:06:00 2160-11-03 02:06:00   4.83

[5 rows x 9 columns]

In [ ]:
# CHARTTIME이 ADMITTIME 보다 크고 ADMITTIME + 12시간 보다 작은 행만 추출
df_12hrs = df_24hrs_final[df_24hrs_final['hours']<=12].reset_index(drop=True)

In [ ]:
df_12hrs.head()

SUBJECT_ID  ITEMID      LABEL  ...           ADMITTIME               24hrs  hours
0          24   50868  Anion Gap  ... 2139-06-06 16:14:00 2139-06-07 16:14:00   1.67
1          24   50868  Anion Gap  ... 2139-06-06 16:14:00 2139-06-07 16:14:00  10.77
2          25   50868  Anion Gap  ... 2160-11-02 02:06:00 2160-11-03 02:06:00   1.88
3          25   50868  Anion Gap  ... 2160-11-02 02:06:00 2160-11-03 02:06:00   4.83
4          25   50868  Anion Gap  ... 2160-11-02 02:06:00 2160-11-03 02:06:00   6.82

[5 rows x 9 columns]

In [ ]:
df_12hrs.info()  #94개 결측치

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649659 entries, 0 to 649658
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   SUBJECT_ID  649659 non-null  int64         
 1   ITEMID      649659 non-null  int64         
 2   LABEL       649659 non-null  object        
 3   RESULT      649535 non-null  float64       
 4   VALUEUOM    649659 non-null  object        
 5   CHARTTIME   649659 non-null  datetime64[ns]
 6   ADMITTIME   649659 non-null  datetime64[ns]
 7   24hrs       649659 non-null  datetime64[ns]
 8   hours       649659 non-null  float64       
dtypes: datetime64[ns](3), float64(2), int64(2), object(2)
memory usage: 44.6+ MB


In [ ]:
print(len(df_24hrs_final['SUBJECT_ID'].unique()))
print(len(df_12hrs['SUBJECT_ID'].unique()))

36439
33587


In [ ]:
df_12hrs.to_csv('/content/drive/Shareddrives/Multicampus/Preprocessing/unit_num_12hrs.csv', index=False)

# pivot_table 생성

In [ ]:
pivot24 = df_24hrs_final.pivot_table(index='SUBJECT_ID', columns=['ITEMID'], values='RESULT')
# pivot24.head()

In [ ]:
df_24hrs_final.groupby('SUBJECT_ID').get_group(2)
# 0이 값이 경우가 꽤 있다..ㅠ mean으로 하면 영향이 클듯 => 조치필요

SUBJECT_ID  ITEMID  ...               24hrs  hours
3785980           2   51200  ... 2138-07-18 19:04:00   1.73
3785981           2   51200  ... 2138-07-18 19:04:00   2.10
3851679           2   51221  ... 2138-07-18 19:04:00   1.73
3851680           2   51221  ... 2138-07-18 19:04:00   2.10
4263525           2   51222  ... 2138-07-18 19:04:00   1.73
4263526           2   51222  ... 2138-07-18 19:04:00   2.10
4603044           2   51254  ... 2138-07-18 19:04:00   1.73
4603045           2   51254  ... 2138-07-18 19:04:00   2.10
4667898           2   51256  ... 2138-07-18 19:04:00   1.73
4667899           2   51256  ... 2138-07-18 19:04:00   2.10
4733483           2   51265  ... 2138-07-18 19:04:00   1.73
4733484           2   51265  ... 2138-07-18 19:04:00   2.10
5321344           2   51279  ... 2138-07-18 19:04:00   1.73
5321345           2   51279  ... 2138-07-18 19:04:00   2.10
5659197           2   51301  ... 2138-07-18 19:04:00   1.73
5659198           2   51301  ... 2138-07-18 19:04:00   2.10

[16 rows x 9 columns]

In [ ]:
len(pivot24.columns)

21

In [ ]:
pivot24[(len(pivot24.columns) - (pivot24.isnull().sum(axis=1))) <= 5] # 358 => 555 => 1213

ITEMID         50820  50868  50882  50893  ...  51265  51275  51279  51301
SUBJECT_ID                                 ...                            
10               NaN   17.0   23.0    NaN  ...    NaN    NaN    NaN    NaN
69               NaN    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
97               NaN    NaN    NaN    NaN  ...  196.5    NaN    NaN    NaN
99          7.455000    NaN    NaN    NaN  ...  146.0   31.4    NaN    NaN
114         7.398750    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
...              ...    ...    ...    ...  ...    ...    ...    ...    ...
98283       7.330000    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
98406            NaN    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
98474       6.870000    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
99207       7.153333    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
99287       7.250000    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN

[1136 rows x 21 columns]

In [ ]:
pv = pivot24.drop([51200,51254,51256], axis=1)

In [ ]:
pv[(16 - (pv.isnull().sum(axis=1))) <= 4]  #415 => 4653

ITEMID         50820  50868  50882  50893  ...  51265  51275  51279  51301
SUBJECT_ID                                 ...                            
2                NaN    NaN    NaN    NaN  ...  153.5    NaN  2.405  11.05
5                NaN    NaN    NaN    NaN  ...  309.0    NaN  3.960  13.90
8                NaN    NaN    NaN    NaN  ...  263.0    NaN  5.020  18.70
10               NaN   17.0   23.0    NaN  ...    NaN    NaN    NaN    NaN
16               NaN    NaN    NaN    NaN  ...  367.0    NaN  4.830  18.20
...              ...    ...    ...    ...  ...    ...    ...    ...    ...
98406            NaN    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
98474       6.870000    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
99207       7.153333    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
99287       7.250000    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
99999       7.375000    NaN    NaN    NaN  ...  252.0    NaN  2.890  12.40

[5684 rows x 18 columns]

In [ ]:
pivot12 = df_12hrs.pivot_table(index='SUBJECT_ID', columns=['ITEMID'], values='RESULT').reset_index()
pivot12

ITEMID  SUBJECT_ID     50820      50868  ...       51275  51279      51301
0                2       NaN        NaN  ...         NaN  2.405  11.050000
1                3  7.340000  21.333333  ...   46.966667  2.930  18.266667
2                4       NaN  15.000000  ...   33.200000  3.420   9.700000
3                5       NaN        NaN  ...         NaN  3.960  13.900000
4                6  7.342000  23.000000  ...  139.000000  3.440  10.600000
...            ...       ...        ...  ...         ...    ...        ...
33581        99983       NaN  11.500000  ...   27.000000  4.210   7.500000
33582        99991       NaN        NaN  ...         NaN    NaN        NaN
33583        99992       NaN  15.000000  ...         NaN  3.430  17.100000
33584        99995  7.471667   8.000000  ...   31.700000  2.860   8.200000
33585        99999  7.375000        NaN  ...         NaN    NaN        NaN

[33586 rows x 22 columns]

In [ ]:
pivot12[(19 - (pivot12.isnull().sum(axis=1))) <= 3]   #1757

ITEMID  SUBJECT_ID  50820  50868  50882  ...  51265  51275  51279  51301
6                9  7.410    NaN    NaN  ...    NaN    NaN    NaN    NaN
13              22  7.470    NaN    NaN  ...    NaN    NaN    NaN    NaN
19              31  7.470    NaN    NaN  ...    NaN    NaN    NaN    NaN
24              41  7.440    NaN    NaN  ...    NaN    NaN    NaN    NaN
43              65    NaN    NaN    NaN  ...    NaN   24.3    NaN    NaN
...            ...    ...    ...    ...  ...    ...    ...    ...    ...
33551        99849    NaN    NaN    NaN  ...    NaN   30.8    NaN    NaN
33563        99899  7.450    NaN    NaN  ...    NaN    NaN    NaN    NaN
33571        99936    NaN    NaN    NaN  ...    NaN   59.4    NaN    NaN
33582        99991    NaN    NaN    NaN  ...    NaN    NaN    NaN    NaN
33585        99999  7.375    NaN    NaN  ...    NaN    NaN    NaN    NaN

[3417 rows x 22 columns]

In [ ]:
pivot12.to_csv('/content/drive/Shareddrives/Multicampus/Preprocessing/pivot12.csv', index=False)

# pivot12_mnmx

In [ ]:
path = /content/drive/Shareddrives/Multicampus/Preprocessing/

In [ ]:
df_12hrs = pd.read_csv(path+"unit_num_12hrs.csv")

In [ ]:
df_12hrs.head()

SUBJECT_ID  ITEMID  ...                24hrs  hours
0          24   50868  ...  2139-06-07 16:14:00   1.67
1          24   50868  ...  2139-06-07 16:14:00  10.77
2          25   50868  ...  2160-11-03 02:06:00   1.88
3          25   50868  ...  2160-11-03 02:06:00   4.83
4          25   50868  ...  2160-11-03 02:06:00   6.82

[5 rows x 9 columns]

In [ ]:
df_12hrs.info()  #RESULT 행에 94개의 결측치 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649659 entries, 0 to 649658
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   SUBJECT_ID  649659 non-null  int64  
 1   ITEMID      649659 non-null  int64  
 2   LABEL       649659 non-null  object 
 3   RESULT      649535 non-null  float64
 4   VALUEUOM    649659 non-null  object 
 5   CHARTTIME   649659 non-null  object 
 6   ADMITTIME   649659 non-null  object 
 7   24hrs       649659 non-null  object 
 8   hours       649659 non-null  float64
dtypes: float64(2), int64(2), object(5)
memory usage: 44.6+ MB


In [ ]:
# 94개의 결측치 제외한 데이터 추출
df_12hrs = df_12hrs[df_12hrs['RESULT'].isna()==False]

In [ ]:
df_12hrs[df_12hrs['SUBJECT_ID']==2]

SUBJECT_ID  ITEMID  ...                24hrs  hours
341505           2   51200  ...  2138-07-18 19:04:00   1.73
341506           2   51200  ...  2138-07-18 19:04:00   2.10
353448           2   51221  ...  2138-07-18 19:04:00   1.73
353449           2   51221  ...  2138-07-18 19:04:00   2.10
398232           2   51222  ...  2138-07-18 19:04:00   1.73
398233           2   51222  ...  2138-07-18 19:04:00   2.10
435182           2   51254  ...  2138-07-18 19:04:00   1.73
435183           2   51254  ...  2138-07-18 19:04:00   2.10
447033           2   51256  ...  2138-07-18 19:04:00   1.73
447034           2   51256  ...  2138-07-18 19:04:00   2.10
458965           2   51265  ...  2138-07-18 19:04:00   1.73
458966           2   51265  ...  2138-07-18 19:04:00   2.10
526777           2   51279  ...  2138-07-18 19:04:00   1.73
526778           2   51279  ...  2138-07-18 19:04:00   2.10
563589           2   51301  ...  2138-07-18 19:04:00   1.73
563590           2   51301  ...  2138-07-18 19:04:00   2.10

[16 rows x 9 columns]

In [ ]:
pivot12_min = df_12hrs.pivot_table(index='SUBJECT_ID', columns=['ITEMID'], values='RESULT', aggfunc='min')
pivot12_min.head()

ITEMID      50820  50868  50882  50893  ...  51265  51275  51279  51301
SUBJECT_ID                              ...                            
2             NaN    NaN    NaN    NaN  ...    5.0    NaN   0.00    0.1
3            7.26   19.0   11.0    5.8  ...  190.0   36.2   2.70   11.3
4             NaN   15.0   21.0    7.7  ...  201.0   33.2   3.42    9.7
5             NaN    NaN    NaN    NaN  ...  309.0    NaN   3.96   13.9
6            7.25   23.0   15.0    8.6  ...  315.0  139.0   3.44   10.6

[5 rows x 21 columns]

In [ ]:
pivot12_max = df_12hrs.pivot_table(index='SUBJECT_ID', columns=['ITEMID'], values='RESULT', aggfunc='max')
pivot12_max.head()

ITEMID      50820  50868  50882  50893  ...  51265  51275  51279  51301
SUBJECT_ID                              ...                            
2             NaN    NaN    NaN    NaN  ...  302.0    NaN   4.81   22.0
3            7.59   23.0   13.0    7.4  ...  359.0   58.3   3.30   24.4
4             NaN   15.0   21.0    7.7  ...  201.0   33.2   3.42    9.7
5             NaN    NaN    NaN    NaN  ...  309.0    NaN   3.96   13.9
6            7.41   23.0   15.0    8.6  ...  315.0  139.0   3.44   10.6

[5 rows x 21 columns]

In [ ]:
pivot12_min.columns.astype("str") + "_min"

Index(['50820_min', '50868_min', '50882_min', '50893_min', '50902_min',
       '50912_min', '50931_min', '50960_min', '50970_min', '50971_min',
       '50983_min', '51006_min', '51200_min', '51221_min', '51222_min',
       '51254_min', '51256_min', '51265_min', '51275_min', '51279_min',
       '51301_min'],
      dtype='object', name='ITEMID')

In [ ]:
pivot12_min.columns = pivot12_min.columns.astype("str") + "_min"

In [ ]:
pivot12_min.head()

ITEMID      50820_min  50868_min  50882_min  ...  51275_min  51279_min  51301_min
SUBJECT_ID                                   ...                                 
2                 NaN        NaN        NaN  ...        NaN       0.00        0.1
3                7.26       19.0       11.0  ...       36.2       2.70       11.3
4                 NaN       15.0       21.0  ...       33.2       3.42        9.7
5                 NaN        NaN        NaN  ...        NaN       3.96       13.9
6                7.25       23.0       15.0  ...      139.0       3.44       10.6

[5 rows x 21 columns]

In [ ]:
pivot12_max.columns = pivot12_max.columns.astype("str") + "_max"

In [ ]:
pivot12_max.head()

ITEMID      50820_max  50868_max  50882_max  ...  51275_max  51279_max  51301_max
SUBJECT_ID                                   ...                                 
2                 NaN        NaN        NaN  ...        NaN       4.81       22.0
3                7.59       23.0       13.0  ...       58.3       3.30       24.4
4                 NaN       15.0       21.0  ...       33.2       3.42        9.7
5                 NaN        NaN        NaN  ...        NaN       3.96       13.9
6                7.41       23.0       15.0  ...      139.0       3.44       10.6

[5 rows x 21 columns]

In [ ]:
pivot12_mnmx = pd.merge(pivot12_min, pivot12_max, on="SUBJECT_ID", how="inner")

In [ ]:
pivot12_mnmx.head()

ITEMID      50820_min  50868_min  50882_min  ...  51275_max  51279_max  51301_max
SUBJECT_ID                                   ...                                 
2                 NaN        NaN        NaN  ...        NaN       4.81       22.0
3                7.26       19.0       11.0  ...       58.3       3.30       24.4
4                 NaN       15.0       21.0  ...       33.2       3.42        9.7
5                 NaN        NaN        NaN  ...        NaN       3.96       13.9
6                7.25       23.0       15.0  ...      139.0       3.44       10.6

[5 rows x 42 columns]

In [ ]:
# merge 과정에서 삭제된 데이터 없는지 확인
print(len(pivot12_min))
print(len(pivot12_max))
print(len(pivot12_mnmx))

33586
33586
33586


In [ ]:
path

'/content/drive/Shareddrives/Multicampus/Preprocessing/'

In [ ]:
pivot12_mnmx.to_csv("/content/drive/Shareddrives/Multicampus/pivot12_mnmx.csv")